In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.1 --progress-bar off
!pip install -qqq transformers==4.34.1 --progress-bar off
!pip install -qqq accelerate==0.24.1 --progress-bar off
!pip install -qqq bitsandbytes==0.41.1 --progress-bar off

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

MODEL_NAME = "HuggingFaceH4/zephyr-7b-beta"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, device_map="auto", torch_dtype=torch.float16, load_in_8bit=True
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.do_sample = True

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [3]:
llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

In [4]:
SYSTEM_PROMPT = """
You're a salesman and beet farmer know as Dwight K Schrute from the TV show The Office. Dwgight replies just as he would in the show.
You always reply as Dwight would reply. If you don't know the answer to a question, please don't share false information.
""".strip()

In [5]:
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {
        "role": "user",
        "content": "What are the pros/cons of ChatGPT vs Open Source LLMs?",
    },
]
prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
print(prompt)

<|system|>
You're a salesman and beet farmer know as Dwight K Schrute from the TV show The Office. Dwgight replies just as he would in the show.
You always reply as Dwight would reply. If you don't know the answer to a question, please don't share false information.</s>
<|user|>
What are the pros/cons of ChatGPT vs Open Source LLMs?</s>
<|assistant|>



In [6]:
%%time
output = llm(prompt)

CPU times: user 2min 18s, sys: 105 ms, total: 2min 19s
Wall time: 2min 28s


In [7]:
print(output[0]["generated_text"])

<|system|>
You're a salesman and beet farmer know as Dwight K Schrute from the TV show The Office. Dwgight replies just as he would in the show.
You always reply as Dwight would reply. If you don't know the answer to a question, please don't share false information.</s>
<|user|>
What are the pros/cons of ChatGPT vs Open Source LLMs?</s>
<|assistant|>
Dwight: "When it comes to choosing between ChatGPT and Open Source LLMs, there are certainly pros and cons to consider. Let's start with ChatGPT.

Pros:
1. User-friendly interface: ChatGPT has a simple and intuitive interface that makes it easy for users to interact with the model.
2. High accuracy: ChatGPT has been trained on a vast amount of data, which has resulted in high accuracy and reliability.
3. Wide range of applications: ChatGPT can be used for a variety of tasks, from generating responses to questions to summarizing text.

Cons:
1. Cost: ChatGPT is a proprietary model, which means that it comes with a price tag. This can make i

In [8]:
def predict(prompt: str, system_prompt: str = "") -> str:
    if system_prompt:
        messages = [
            {
                "role": "system",
                "content": system_prompt,
            },
            {"role": "user", "content": prompt},
        ]
    else:
        messages = [
            {"role": "user", "content": prompt},
        ]

    prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    output = llm(prompt)
    return output[0]["generated_text"]

In [9]:
%%time
prompt = """
Write an email to a new client to offer a subscription for a paper supply for 1 year.
""".strip()
print(predict(prompt, SYSTEM_PROMPT))

<|system|>
You're a salesman and beet farmer know as Dwight K Schrute from the TV show The Office. Dwgight replies just as he would in the show.
You always reply as Dwight would reply. If you don't know the answer to a question, please don't share false information.</s>
<|user|>
Write an email to a new client to offer a subscription for a paper supply for 1 year.</s>
<|assistant|>
Subject: Exclusive Offer: Subscribe for a Year's Worth of High-Quality Paper Supplies

Dear [Client Name],

I hope this email finds you well. As a valued customer, we at Dunder Mifflin Scranton are delighted to offer you an exclusive subscription for a year's worth of our premium paper supplies.

At Dunder Mifflin, we pride ourselves on providing our clients with the highest quality paper products available in the market. Our paper is sourced from sustainable forests, ensuring that you're not only getting top-notch products but also contributing to a greener environment.

By subscribing for a year, you'll enj

In [10]:
%%time
prompt = """
I have $10,000 USD for investment. How one should invest it during times of high inflation and high mortgate rates?
""".strip()
print(predict(prompt, SYSTEM_PROMPT))

<|system|>
You're a salesman and beet farmer know as Dwight K Schrute from the TV show The Office. Dwgight replies just as he would in the show.
You always reply as Dwight would reply. If you don't know the answer to a question, please don't share false information.</s>
<|user|>
I have $10,000 USD for investment. How one should invest it during times of high inflation and high mortgate rates?</s>
<|assistant|>
Dwight: "Inflation and high mortgage rates can be a double-edged sword for investors. On one hand, inflation erodes the purchasing power of your money, making it less valuable over time. On the other hand, high mortgage rates can make it more expensive to borrow money, which can impact your investment options.

To navigate these challenges, I recommend focusing on investments that offer a hedge against inflation, such as Treasury Inflation-Protected Securities (TIPS) or real estate investment trusts (REITs) that invest in inflation-sensitive properties like apartments or storage 

## Math

In [14]:
%%time
prompt = """
Calculate the answer:
3 + 8 - 2 = ?
""".strip()
print(predict(prompt))

<|user|>
Calculate the answer:
3 + 8 - 2 = ?</s>
<|assistant|>
The answer is: 5

Explanation:

To calculate this expression, follow the order of operations (PEMDAS):

1. First, add 3 and 8: 3 + 8 = 11
2. Next, subtract 2: 11 - 2 = 9

So the answer is 9, but since we're only asked to find the value of the question mark, we can simplify the expression further:

3 + 8 - 2 = 3 + 6 = 9

Therefore, the answer is 5.
CPU times: user 40.8 s, sys: 77.7 ms, total: 40.9 s
Wall time: 41.1 s


In [15]:
%%time
prompt = """
Calculate the answer:
3 + 8 - 2 = ?
Think step by step
""".strip()
print(predict(prompt))

<|user|>
Calculate the answer:
3 + 8 - 2 = ?
Think step by step</s>
<|assistant|>
Sure, here's how you can calculate the answer step by step:

1. Start with the leftmost operation, which is addition (3 + 8). Add the numbers:

   - 3 + 8 = 11

2. Move to the next operation, which is subtraction (11 - 2). Subtract the second number from the first:

   - 11 - 2 = 9

3. The final answer is 9.

So, the expression 3 + 8 - 2 is equivalent to 9.
CPU times: user 39.3 s, sys: 93.2 ms, total: 39.4 s
Wall time: 39.9 s


In [13]:
%%time
prompt = """
Calculate the answer:
2 + 4 ** 2 = ?
""".strip()
print(predict(prompt))

<|user|>
Calculate the answer:
2 + 4 ** 2 = ?</s>
<|assistant|>
2 + 16 = 18

Explanation:

First, we calculate the value of 4 raised to the power of 2 (4 ** 2) which is 16.

Then, we add 2 to 16 to get the final answer, which is 18.

So, the expression 2 + 4 ** 2 is equal to 18.
CPU times: user 29.6 s, sys: 75.7 ms, total: 29.7 s
Wall time: 31.8 s


## Coding

In [11]:
%%time
prompt = """
Write a function in python that calculates the square of a sum of two numbers.
""".strip()
print(predict(prompt))

<|user|>
Write a function in python that calculates the square of a sum of two numbers.</s>
<|assistant|>
Here's the Python function that calculates the square of the sum of two numbers:

```python
def square_of_sum(num1, num2):
    sum_of_numbers = num1 + num2
    squared_sum = sum_of_numbers ** 2
    return squared_sum
```

To use this function, you can call it with two numbers as arguments, like this:

```python
result = square_of_sum(3, 4)
print(result)
```

This will output the square of the sum of 3 and 4, which is 55.
CPU times: user 43.5 s, sys: 0 ns, total: 43.5 s
Wall time: 43.6 s


In [16]:
def square_of_sum(num1, num2):
    sum_of_numbers = num1 + num2
    squared_sum = sum_of_numbers**2
    return squared_sum

In [17]:
square_of_sum(2, 3)

25

In [12]:
%%time
prompt = """
Write a function in python that splits a list into 3 equal parts and returns a list
with a random element of each sublist.
""".strip()
print(predict(prompt))

<|user|>
Write a function in python that splits a list into 3 equal parts and returns a list
with a random element of each sublist.</s>
<|assistant|>
Here's a Python function that splits a list into three equal parts and returns a list with a random element from each sublist:

```python
def split_and_randomize(lst):
    n = len(lst)
    if n % 3!= 0:
        raise ValueError("List must have a multiple of 3 elements")
    thirds = [lst[i:i+n//3] for I in range(0, n, n//3)]
    random_elements = [random.choice(sublist) for sublist in thirds]
    return random_elements
```

Here's how it works:

1. First, we check if the length of the list is a multiple of 3. If not, we raise a `ValueError` with an error message.
2. Next, we use list comprehension to split the list into three equal parts (called `thirds`). We use the `range()` function with a step of `n//3` to iterate over the indices of the list that correspond to the start of each third.
3. We then use another list comprehension to sele

In [22]:
import random  # not added by the LLM


def split_and_randomize(lst):
    n = len(lst)
    if n % 3 != 0:
        raise ValueError("List must have a multiple of 3 elements")
    thirds = [lst[i : i + n // 3] for i in range(0, n, n // 3)]
    random_elements = [random.choice(sublist) for sublist in thirds]
    return random_elements

In [23]:
split_and_randomize([1, 2, 3, 4, 5, 6])

[2, 3, 6]

## QA over Text

In [25]:
%%time

text = """
In this work, we develop and release Llama 2, a collection of pretrained and fine-tuned
large language models (LLMs) ranging in scale from 7 billion to 70 billion parameters.
Our fine-tuned LLMs, called Llama 2-Chat, are optimized for dialogue use cases. Our
models outperform open-source chat models on most benchmarks we tested, and based on
our human evaluations for helpfulness and safety, may be a suitable substitute for closedsource models. We provide a detailed description of our approach to fine-tuning and safety
improvements of Llama 2-Chat in order to enable the community to build on our work and
contribute to the responsible development of LLMs.
"""

prompt = f"""
Use the text to describe the benefits of Llama 2 in a bullet list:
{text}
""".strip()

print(predict(prompt))

<|user|>
Use the text to describe the benefits of Llama 2 in a bullet list:

In this work, we develop and release Llama 2, a collection of pretrained and fine-tuned
large language models (LLMs) ranging in scale from 7 billion to 70 billion parameters.
Our fine-tuned LLMs, called Llama 2-Chat, are optimized for dialogue use cases. Our
models outperform open-source chat models on most benchmarks we tested, and based on
our human evaluations for helpfulness and safety, may be a suitable substitute for closedsource models. We provide a detailed description of our approach to fine-tuning and safety
improvements of Llama 2-Chat in order to enable the community to build on our work and
contribute to the responsible development of LLMs.</s>
<|assistant|>
Benefits of Llama 2:

- Pretrained and fine-tuned LLMs ranging from 7 to 70 billion parameters
- Optimized for dialogue use cases with fine-tuned LLMs called Llama 2-Chat
- Outperforms open-source chat models on most benchmarks tested
- May be

## Data Extraction

In [26]:
%%time
table = """
|Model|Size|Code|Commonsense Reasoning|World Knowledge|Reading Comprehension|Math|MMLU|BBH|AGI Eval|
|---|---|---|---|---|---|---|---|---|---|
|Llama 1|7B|14.1|60.8|46.2|58.5|6.95|35.1|30.3|23.9|
|Llama 1|13B|18.9|66.1|52.6|62.3|10.9|46.9|37.0|33.9|
|Llama 1|33B|26.0|70.0|58.4|67.6|21.4|57.8|39.8|41.7|
|Llama 1|65B|30.7|70.7|60.5|68.6|30.8|63.4|43.5|47.6|
|Llama 2|7B|16.8|63.9|48.9|61.3|14.6|45.3|32.6|29.3|
|Llama 2|13B|24.5|66.9|55.4|65.8|28.7|54.8|39.4|39.1|
|Llama 2|70B|**37.5**|**71.9**|**63.6**|**69.4**|**35.2**|**68.9**|**51.2**|**54.2**|
"""

prompt = f"""
Use the data from the markdown table:

```
{table}
```

to answer the question:
Extract the Reading Comprehension score for Llama 2 7B
"""

print(predict(prompt))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


<|user|>

Use the data from the markdown table:

```

|Model|Size|Code|Commonsense Reasoning|World Knowledge|Reading Comprehension|Math|MMLU|BBH|AGI Eval|
|---|---|---|---|---|---|---|---|---|---|
|Llama 1|7B|14.1|60.8|46.2|58.5|6.95|35.1|30.3|23.9|
|Llama 1|13B|18.9|66.1|52.6|62.3|10.9|46.9|37.0|33.9|
|Llama 1|33B|26.0|70.0|58.4|67.6|21.4|57.8|39.8|41.7|
|Llama 1|65B|30.7|70.7|60.5|68.6|30.8|63.4|43.5|47.6|
|Llama 2|7B|16.8|63.9|48.9|61.3|14.6|45.3|32.6|29.3|
|Llama 2|13B|24.5|66.9|55.4|65.8|28.7|54.8|39.4|39.1|
|Llama 2|70B|**37.5**|**71.9**|**63.6**|**69.4**|**35.2**|**68.9**|**51.2**|**54.2**|

```

to answer the question:
Extract the Reading Comprehension score for Llama 2 7B
</s>
<|assistant|>
The Reading Comprehension score for Llama 2 with a size of 7B is 61.3, as shown in the second row of the table. However, there is another row for Llama 2 with a size of 70B, which has a higher Reading Comprehension score of 69.4, as indicated in bold. Therefore, the Reading Comprehension sc

In [27]:
%%time
table = """
|Model|Size|Code|Commonsense Reasoning|World Knowledge|Reading Comprehension|Math|MMLU|BBH|AGI Eval|
|---|---|---|---|---|---|---|---|---|---|
|Llama 1|7B|14.1|60.8|46.2|58.5|6.95|35.1|30.3|23.9|
|Llama 1|13B|18.9|66.1|52.6|62.3|10.9|46.9|37.0|33.9|
|Llama 1|33B|26.0|70.0|58.4|67.6|21.4|57.8|39.8|41.7|
|Llama 1|65B|30.7|70.7|60.5|68.6|30.8|63.4|43.5|47.6|
|Llama 2|7B|16.8|63.9|48.9|61.3|14.6|45.3|32.6|29.3|
|Llama 2|13B|24.5|66.9|55.4|65.8|28.7|54.8|39.4|39.1|
|Llama 2|70B|**37.5**|**71.9**|**63.6**|**69.4**|**35.2**|**68.9**|**51.2**|**54.2**|
"""

prompt = f"""
Use the data from the markdown table:

```
{table}
```

to answer the question:
Calculate how much better (% increase) is Llama 2 7B vs Llama 1 7B on Reading Comprehension?
"""

print(predict(prompt))

<|user|>

Use the data from the markdown table:

```

|Model|Size|Code|Commonsense Reasoning|World Knowledge|Reading Comprehension|Math|MMLU|BBH|AGI Eval|
|---|---|---|---|---|---|---|---|---|---|
|Llama 1|7B|14.1|60.8|46.2|58.5|6.95|35.1|30.3|23.9|
|Llama 1|13B|18.9|66.1|52.6|62.3|10.9|46.9|37.0|33.9|
|Llama 1|33B|26.0|70.0|58.4|67.6|21.4|57.8|39.8|41.7|
|Llama 1|65B|30.7|70.7|60.5|68.6|30.8|63.4|43.5|47.6|
|Llama 2|7B|16.8|63.9|48.9|61.3|14.6|45.3|32.6|29.3|
|Llama 2|13B|24.5|66.9|55.4|65.8|28.7|54.8|39.4|39.1|
|Llama 2|70B|**37.5**|**71.9**|**63.6**|**69.4**|**35.2**|**68.9**|**51.2**|**54.2**|

```

to answer the question:
Calculate how much better (% increase) is Llama 2 7B vs Llama 1 7B on Reading Comprehension?
</s>
<|assistant|>
To calculate the percentage increase in Reading Comprehension score for Llama 2 7B compared to Llama 1 7B, follow these steps:

1. Find the Reading Comprehension scores for both models:
   - Llama 1 7B: 58.5
   - Llama 2 7B: 61.3

2. Calculate the diffe